In [ ]:
#!/usr/bin/env python3
import random
import pandas as pd
from datetime import datetime, timedelta

# -------------------------------
# Helper functions
# -------------------------------
def random_date(start, end):
    """Return a random datetime between start and end."""
    delta = end - start
    random_seconds = random.randint(0, int(delta.total_seconds()))
    return start + timedelta(seconds=random_seconds)

def format_date(dt):
    return dt.strftime("%Y-%m-%d")

def random_dob(elderly=False, update_date=None):
    """Return a random date-of-birth.
       For elderly customers, choose a date between 1930 and (update_date - 65 years).
       Otherwise choose between 1960 and 2000."""
    if elderly and update_date is not None:
        # To guarantee age>=65 at update_date, choose between 1930 and update_date - 65 years.
        max_year = update_date.year - 65
        start_year = 1930
        year = random.randint(start_year, max_year)
        month = random.randint(1, 12)
        day = random.randint(1, 28)
        return datetime(year, month, day)
    else:
        year = random.randint(1960, 2000)
        month = random.randint(1, 12)
        day = random.randint(1, 28)
        return datetime(year, month, day)

def dummy_value(profile_type, cust_num, new=True):
    """Generate a dummy new or old value based on profile type."""
    if profile_type == "EMAIL":
        return f"{'new' if new else 'old'}{cust_num}@example.com"
    elif profile_type == "PHONE":
        # Generate a dummy US-style phone number.
        return f"555-{random.randint(100,999)}-{random.randint(1000,9999)}"
    elif profile_type == "HOME ADDRESS":
        return f"{random.randint(10,9999)} {'New' if new else 'Old'} St., SomeCity"
    else:
        return "N/A"

# -------------------------------
# Settings for sample data generation
# -------------------------------
random.seed(42)  # for reproducibility

# We choose our profile update dates to lie in December 2024.
update_date_start = datetime(2024, 12, 1)
update_date_end   = datetime(2024, 12, 31, 23, 59, 59)

# Define scenario counts (total profile_change records = 200)
counts = {
    "clear_fraud": 40,
    "structuring": 10,   # these will get 3 transactions each
    "collusion": 20,     # 20 customers with 2 updates each => 40 records
    "legacy": 20,
    "employee_workflow": 10,  # all by a designated employee (whitelisted)
    "no_alert": 80
}

# List of available employee IDs for most scenarios (avoid "E999" which we reserve)
employee_ids = [f"E{str(i).zfill(3)}" for i in range(1, 21)]
# For employee_workflow, we use a dedicated employee:
workflow_emp = "E999"

# -------------------------------
# Generate Profile Changes Data
# -------------------------------
profile_changes = []
next_cust_num = 1000  # start customer numbering

# For collusion, we want a customer to have 2 records (with different CHG_BY)
collusion_customers = []  # will hold tuples: (cust_num, update_dates, [emp_ids])
# --- Clear Fraud (must be elderly) ---
for _ in range(counts["clear_fraud"]):
    cust_num = next_cust_num
    next_cust_num += 1
    # generate an update date in December 2024
    update_dt = random_date(update_date_start, update_date_end)
    dob = random_dob(elderly=True, update_date=update_dt)
    # choose a contact type randomly among EMAIL, PHONE, HOME ADDRESS
    profile = random.choice(["EMAIL", "PHONE", "HOME ADDRESS"])
    profile_changes.append({
        "CUST_NUM": cust_num,
        "DATE_OF_BIRTH": format_date(dob),
        "CUST_TYPE": 0,  # assume individual
        "PROFILE_UPDATE_DATE": format_date(update_dt),
        "PROFILE": profile,
        "NEW_VALUE": dummy_value(profile, cust_num, new=True),
        "OLD_VALUE": dummy_value(profile, cust_num, new=False),
        "CHG_BY": random.choice(employee_ids),
        "JOB_TITLE": "CSR",
        "EMPLOYEE_JOB_TITLE": "CSR",
        "scenario": "clear_fraud"
    })

# --- Structuring (will generate 3 transaction records later) ---
for _ in range(counts["structuring"]):
    cust_num = next_cust_num
    next_cust_num += 1
    update_dt = random_date(update_date_start, update_date_end)
    dob = random_dob(elderly=False)  # not necessarily elderly
    profile = random.choice(["EMAIL", "PHONE", "HOME ADDRESS"])
    profile_changes.append({
        "CUST_NUM": cust_num,
        "DATE_OF_BIRTH": format_date(dob),
        "CUST_TYPE": 0,
        "PROFILE_UPDATE_DATE": format_date(update_dt),
        "PROFILE": profile,
        "NEW_VALUE": dummy_value(profile, cust_num, new=True),
        "OLD_VALUE": dummy_value(profile, cust_num, new=False),
        "CHG_BY": random.choice(employee_ids),
        "JOB_TITLE": "CSR",
        "EMPLOYEE_JOB_TITLE": "CSR",
        "scenario": "structuring"
    })

# --- Collusion (each customer appears twice with different employees) ---
for _ in range(counts["collusion"]):
    cust_num = next_cust_num
    next_cust_num += 1
    # Generate two updates for this same customer:
    # Pick two different employees
    emp_pair = random.sample(employee_ids, 2)
    for emp in emp_pair:
        update_dt = random_date(update_date_start, update_date_end)
        dob = random_dob(elderly=False)
        profile = random.choice(["EMAIL", "PHONE", "HOME ADDRESS"])
        profile_changes.append({
            "CUST_NUM": cust_num,
            "DATE_OF_BIRTH": format_date(dob),
            "CUST_TYPE": 0,
            "PROFILE_UPDATE_DATE": format_date(update_dt),
            "PROFILE": profile,
            "NEW_VALUE": dummy_value(profile, cust_num, new=True),
            "OLD_VALUE": dummy_value(profile, cust_num, new=False),
            "CHG_BY": emp,
            "JOB_TITLE": "CSR",
            "EMPLOYEE_JOB_TITLE": "CSR",
            "scenario": "collusion"
        })

# --- Legacy Customer (large legitimate transaction, no fraud signals) ---
for _ in range(counts["legacy"]):
    cust_num = next_cust_num
    next_cust_num += 1
    update_dt = random_date(update_date_start, update_date_end)
    dob = random_dob(elderly=False)
    profile = random.choice(["EMAIL", "PHONE", "HOME ADDRESS"])
    profile_changes.append({
        "CUST_NUM": cust_num,
        "DATE_OF_BIRTH": format_date(dob),
        "CUST_TYPE": 0,
        "PROFILE_UPDATE_DATE": format_date(update_dt),
        "PROFILE": profile,
        "NEW_VALUE": dummy_value(profile, cust_num, new=True),
        "OLD_VALUE": dummy_value(profile, cust_num, new=False),
        "CHG_BY": random.choice(employee_ids),
        "JOB_TITLE": "CSR",
        "EMPLOYEE_JOB_TITLE": "CSR",
        "scenario": "legacy"
    })

# --- Employee Workflow (whitelisted) ---
for _ in range(counts["employee_workflow"]):
    cust_num = next_cust_num
    next_cust_num += 1
    update_dt = random_date(update_date_start, update_date_end)
    dob = random_dob(elderly=False)
    # For this scenario we force the update to be HOME ADDRESS
    profile = "HOME ADDRESS"
    profile_changes.append({
        "CUST_NUM": cust_num,
        "DATE_OF_BIRTH": format_date(dob),
        "CUST_TYPE": 0,
        "PROFILE_UPDATE_DATE": format_date(update_dt),
        "PROFILE": profile,
        "NEW_VALUE": dummy_value(profile, cust_num, new=True),
        "OLD_VALUE": dummy_value(profile, cust_num, new=False),
        "CHG_BY": workflow_emp,
        "JOB_TITLE": "CSR",
        "EMPLOYEE_JOB_TITLE": "CSR",
        "scenario": "employee_workflow"
    })

# --- No Alert (normal update with benign transaction) ---
for _ in range(counts["no_alert"]):
    cust_num = next_cust_num
    next_cust_num += 1
    update_dt = random_date(update_date_start, update_date_end)
    dob = random_dob(elderly=False)
    profile = random.choice(["EMAIL", "PHONE", "HOME ADDRESS"])
    profile_changes.append({
        "CUST_NUM": cust_num,
        "DATE_OF_BIRTH": format_date(dob),
        "CUST_TYPE": 0,
        "PROFILE_UPDATE_DATE": format_date(update_dt),
        "PROFILE": profile,
        "NEW_VALUE": dummy_value(profile, cust_num, new=True),
        "OLD_VALUE": dummy_value(profile, cust_num, new=False),
        "CHG_BY": random.choice(employee_ids),
        "JOB_TITLE": "CSR",
        "EMPLOYEE_JOB_TITLE": "CSR",
        "scenario": "no_alert"
    })

# Convert to DataFrame and (optionally) drop the internal 'scenario' column before saving.
profile_df = pd.DataFrame(profile_changes)
profile_df_to_save = profile_df.drop(columns=["scenario"])
profile_df_to_save = profile_df_to_save.sort_values("PROFILE_UPDATE_DATE").reset_index(drop=True)

# Write to CSV file.
profile_df_to_save.to_csv("profile_changes.csv", index=False)
print("Generated 'profile_changes.csv' with", len(profile_df_to_save), "records.")

# -------------------------------
# Generate Transaction History Data
# -------------------------------
# For each profile change record, we generate transactions in the window [update_dt, update_dt+3days].
transactions = []

# For each row in our profile_changes list, use the stored scenario.
for _, row in profile_df.iterrows():
    cust_num = row["CUST_NUM"]
    # Create an account number based on the customer number (for simplicity)
    acct_num = f"ACCT{cust_num}"
    # Parse the PROFILE_UPDATE_DATE back to a datetime object.
    update_dt = datetime.strptime(row["PROFILE_UPDATE_DATE"], "%Y-%m-%d")
    window_start = update_dt
    window_end = update_dt + timedelta(days=3)
    
    # Choose a random transaction date within the window.
    def tx_date():
        return random_date(window_start, window_end).strftime("%Y-%m-%d")
    
    scenario = row["scenario"]
    
    if scenario == "clear_fraud":
        # One transaction: a large withdrawal (negative amount ≤ -10000)
        amount = -random.randint(10000, 20000)
        transactions.append({
            "CUST_NUM": cust_num,
            "ACCT_NUM": acct_num,
            "TRANSACTION_DATE": tx_date(),
            "TRANSACTION_AMOUNT": amount,
            "TRANSACTION_DESCRIPTION": "Large Withdrawal Clear Fraud"
        })
    elif scenario == "structuring":
        # Three transactions: small withdrawals between -500 and -9999.
        for _ in range(3):
            amount = -random.randint(500, 9999)
            transactions.append({
                "CUST_NUM": cust_num,
                "ACCT_NUM": acct_num,
                "TRANSACTION_DATE": tx_date(),
                "TRANSACTION_AMOUNT": amount,
                "TRANSACTION_DESCRIPTION": "Small Withdrawal Structuring"
            })
    elif scenario == "collusion":
        # One normal transaction.
        amount = -random.randint(100, 500)  # small withdrawal (or could be deposit)
        transactions.append({
            "CUST_NUM": cust_num,
            "ACCT_NUM": acct_num,
            "TRANSACTION_DATE": tx_date(),
            "TRANSACTION_AMOUNT": amount,
            "TRANSACTION_DESCRIPTION": "Normal Transaction Collusion"
        })
    elif scenario == "legacy":
        # One transaction: large deposit (to simulate legitimate activity)
        amount = random.randint(10000, 20000)
        transactions.append({
            "CUST_NUM": cust_num,
            "ACCT_NUM": acct_num,
            "TRANSACTION_DATE": tx_date(),
            "TRANSACTION_AMOUNT": amount,
            "TRANSACTION_DESCRIPTION": "Large Legitimate Transaction"
        })
    elif scenario == "employee_workflow":
        # One benign transaction.
        amount = random.randint(100, 500)
        transactions.append({
            "CUST_NUM": cust_num,
            "ACCT_NUM": acct_num,
            "TRANSACTION_DATE": tx_date(),
            "TRANSACTION_AMOUNT": amount,
            "TRANSACTION_DESCRIPTION": "Normal Transaction Employee Workflow"
        })
    elif scenario == "no_alert":
        # One normal transaction that should not trigger any rule.
        amount = random.choice([random.randint(50, 300), -random.randint(50, 300)])
        transactions.append({
            "CUST_NUM": cust_num,
            "ACCT_NUM": acct_num,
            "TRANSACTION_DATE": tx_date(),
            "TRANSACTION_AMOUNT": amount,
            "TRANSACTION_DESCRIPTION": "Normal Transaction"
        })

# Convert transactions to DataFrame.
txn_df = pd.DataFrame(transactions)
# (Optionally, sort by TRANSACTION_DATE.)
txn_df = txn_df.sort_values("TRANSACTION_DATE").reset_index(drop=True)

# Write to CSV file.
txn_df.to_csv("transaction_history.csv", index=False)
print("Generated 'transaction_history.csv' with", len(txn_df), "records.")


In [9]:
#!/usr/bin/env python3
import pandas as pd
from datetime import timedelta

def main():
    # ====================================================
    # STEP 1. LOAD THE DATASETS
    # ----------------------------------------------------
    profile_file = 'profile_changes.csv'
    txn_file     = 'transaction_history.csv'
    
    # Load the profile changes data.
    # Expected columns include:
    #   CUST_NUM, DATE_OF_BIRTH, CUST_TYPE, PROFILE_UPDATE_DATE,
    #   PROFILE (e.g., EMAIL, PHONE, or HOME ADDRESS), NEW_VALUE, OLD_VALUE,
    #   CHG_BY, JOB_TITLE, EMPLOYEE_JOB_TITLE, etc.
    profile_df = pd.read_csv(profile_file, parse_dates=['DATE_OF_BIRTH', 'PROFILE_UPDATE_DATE'])
    
    # Load the transaction history data.
    # Expected columns include:
    #   CUST_NUM, ACCT_NUM, TRANSACTION_DATE, TRANSACTION_AMOUNT, TRANSACTION_DESCRIPTION.
    txn_df = pd.read_csv(txn_file, parse_dates=['TRANSACTION_DATE'])
    
    # ====================================================
    # STEP 2. PRE-PROCESSING
    # ----------------------------------------------------
    # Compute customer's age at profile update.
    profile_df['AGE'] = profile_df.apply(
        lambda row: (row['PROFILE_UPDATE_DATE'] - row['DATE_OF_BIRTH']).days // 365, axis=1
    )
    
    # Compute ISO week number (for grouping employee activity).
    profile_df['WEEK'] = profile_df['PROFILE_UPDATE_DATE'].dt.isocalendar().week
    
    # Define the post-update transaction window (3 days after the profile update).
    profile_df['window_start'] = profile_df['PROFILE_UPDATE_DATE']
    profile_df['window_end']   = profile_df['PROFILE_UPDATE_DATE'] + pd.Timedelta(days=3)
    
    # Standardize the PROFILE field.
    profile_df['PROFILE_UPPER'] = profile_df['PROFILE'].str.strip().str.upper()
    
    # ====================================================
    # STEP 3. DEFINE THRESHOLDS & CRITERIA
    # ----------------------------------------------------
    ELDERLY_AGE = 65
    LARGE_WITHDRAWAL_THRESHOLD = 10000   # For Clear Fraud: withdrawal amounts <= -$10,000.
    SMALL_WITHDRAWAL_THRESHOLD = 500     # Lower bound for small withdrawals (<= -$500).
    SMALL_WITHDRAWAL_COUNT_THRESHOLD = 3 # For Structuring: at least 3 small withdrawals.
    EMPLOYEE_HOME_ADDRESS_THRESHOLD = 5  # For Employee Workflow: 5+ HOME ADDRESS updates in a week.
    
    # ====================================================
    # STEP 4. IDENTIFY WHITELISTED EMPLOYEES (Employee Workflow)
    # ----------------------------------------------------
    # Employees updating many HOME ADDRESS records in a given week are whitelisted.
    home_address_df = profile_df[profile_df['PROFILE_UPPER'] == 'HOME ADDRESS']
    emp_week_counts = home_address_df.groupby(['CHG_BY', 'WEEK']).size().reset_index(name='count')
    whitelisted_emp_week = set(
        tuple(x) for x in emp_week_counts[emp_week_counts['count'] >= EMPLOYEE_HOME_ADDRESS_THRESHOLD][['CHG_BY', 'WEEK']].to_numpy()
    )
    
    # ====================================================
    # STEP 5. PROCESS EACH PROFILE RECORD:
    #         - Compute pre (21 days) and post (3 days) transaction summaries.
    #         - Detect fraud scenarios (including new Account Take Over conditions).
    #         - Build detailed reasoning for each record.
    # ----------------------------------------------------
    audit_alerts = []   # To store alert(s) for each record.
    reasoning_list = [] # To store detailed reasoning messages.
    
    # Lists for transaction summary statistics.
    pre_txn_count_list       = []
    pre_txn_total_list       = []
    pre_txn_avg_list         = []
    pre_txn_deposits_list    = []
    pre_txn_withdrawals_list = []
    pre_txn_details_list     = []
    
    post_txn_count_list       = []
    post_txn_total_list       = []
    post_txn_avg_list         = []
    post_txn_deposits_list    = []
    post_txn_withdrawals_list = []
    post_txn_details_list     = []
    
    for idx, row in profile_df.iterrows():
        cust_num     = row['CUST_NUM']
        update_dt    = row['PROFILE_UPDATE_DATE']
        week         = row['WEEK']
        employee     = row['CHG_BY']
        profile_type = row['PROFILE_UPPER']
        age          = row['AGE']
        # For ATO scenario, check employee job title.
        emp_job_title = row['EMPLOYEE_JOB_TITLE'] if 'EMPLOYEE_JOB_TITLE' in row and pd.notnull(row['EMPLOYEE_JOB_TITLE']) else "CSR"
        
        # ----- Define Transaction Windows -----
        # Pre-update window: 21 days prior to update (excluding update date).
        pre_window_start = update_dt - pd.Timedelta(days=21)
        pre_window_end   = update_dt
        pre_txns = txn_df[
            (txn_df['CUST_NUM'] == cust_num) &
            (txn_df['TRANSACTION_DATE'] >= pre_window_start) &
            (txn_df['TRANSACTION_DATE'] < pre_window_end)
        ]
        
        # Post-update window: update date through next 3 days.
        post_window_start = update_dt
        post_window_end   = update_dt + pd.Timedelta(days=3)
        post_txns = txn_df[
            (txn_df['CUST_NUM'] == cust_num) &
            (txn_df['TRANSACTION_DATE'] >= post_window_start) &
            (txn_df['TRANSACTION_DATE'] <= post_window_end)
        ]
        
        # ----- Compute Pre-Transaction Summary Stats -----
        pre_count       = pre_txns.shape[0]
        pre_total       = pre_txns['TRANSACTION_AMOUNT'].sum() if pre_count > 0 else 0
        pre_avg         = pre_txns['TRANSACTION_AMOUNT'].mean() if pre_count > 0 else 0
        pre_deposits    = pre_txns[pre_txns['TRANSACTION_AMOUNT'] > 0].shape[0]
        pre_withdrawals = pre_txns[pre_txns['TRANSACTION_AMOUNT'] < 0].shape[0]
        if pre_count > 0:
            pre_details = "; ".join(
                pre_txns.apply(lambda x: f"{x['TRANSACTION_DATE'].strftime('%Y-%m-%d')}({x['TRANSACTION_AMOUNT']})", axis=1)
            )
        else:
            pre_details = ""
        
        pre_txn_count_list.append(pre_count)
        pre_txn_total_list.append(pre_total)
        pre_txn_avg_list.append(pre_avg)
        pre_txn_deposits_list.append(pre_deposits)
        pre_txn_withdrawals_list.append(pre_withdrawals)
        pre_txn_details_list.append(pre_details)
        
        # ----- Compute Post-Transaction Summary Stats -----
        post_count       = post_txns.shape[0]
        post_total       = post_txns['TRANSACTION_AMOUNT'].sum() if post_count > 0 else 0
        post_avg         = post_txns['TRANSACTION_AMOUNT'].mean() if post_count > 0 else 0
        post_deposits    = post_txns[post_txns['TRANSACTION_AMOUNT'] > 0].shape[0]
        post_withdrawals = post_txns[post_txns['TRANSACTION_AMOUNT'] < 0].shape[0]
        if post_count > 0:
            post_details = "; ".join(
                post_txns.apply(lambda x: f"{x['TRANSACTION_DATE'].strftime('%Y-%m-%d')}({x['TRANSACTION_AMOUNT']})", axis=1)
            )
        else:
            post_details = ""
        
        post_txn_count_list.append(post_count)
        post_txn_total_list.append(post_total)
        post_txn_avg_list.append(post_avg)
        post_txn_deposits_list.append(post_deposits)
        post_txn_withdrawals_list.append(post_withdrawals)
        post_txn_details_list.append(post_details)
        
        # ----- Detect Fraud Scenarios & Build Reasoning -----
        alerts = []
        record_reasoning = []  # To accumulate reasoning messages.
        
        # 1. Employee Workflow (Whitelisted)
        if (employee, week) in whitelisted_emp_week:
            alerts.append(("Employee Workflow", "Whitelisted"))
            emp_count = emp_week_counts[(emp_week_counts['CHG_BY'] == employee) & (emp_week_counts['WEEK'] == week)]['count'].values[0]
            record_reasoning.append(
                f"Employee Workflow: Employee {employee} performed {emp_count} HOME ADDRESS updates in week {week} (threshold = {EMPLOYEE_HOME_ADDRESS_THRESHOLD})."
            )
        else:
            # 2. Clear Fraud: Elderly (>=65) + Contact Change + Large Withdrawal (post-update)
            if age >= ELDERLY_AGE and profile_type in ['EMAIL', 'PHONE', 'HOME ADDRESS']:
                large_withdrawals = post_txns[post_txns['TRANSACTION_AMOUNT'] <= -LARGE_WITHDRAWAL_THRESHOLD]
                if not large_withdrawals.empty:
                    alerts.append(("Clear Fraud", "High Risk Score"))
                    lw_details = "; ".join(
                        large_withdrawals.apply(lambda x: f"{x['TRANSACTION_DATE'].strftime('%Y-%m-%d')}({x['TRANSACTION_AMOUNT']})", axis=1)
                    )
                    record_reasoning.append(
                        f"Clear Fraud: Elderly customer (age {age}) with {profile_type} update made large withdrawal(s): {lw_details}."
                    )
            # 3. Structuring: Multiple small withdrawals post-update.
            small_withdrawals = post_txns[
                (post_txns['TRANSACTION_AMOUNT'] <= -SMALL_WITHDRAWAL_THRESHOLD) &
                (post_txns['TRANSACTION_AMOUNT'] > -LARGE_WITHDRAWAL_THRESHOLD)
            ]
            if small_withdrawals.shape[0] >= SMALL_WITHDRAWAL_COUNT_THRESHOLD:
                alerts.append(("Structuring", "Medium Risk Score"))
                sw_details = "; ".join(
                    small_withdrawals.apply(lambda x: f"{x['TRANSACTION_DATE'].strftime('%Y-%m-%d')}({x['TRANSACTION_AMOUNT']})", axis=1)
                )
                record_reasoning.append(
                    f"Structuring: Found {small_withdrawals.shape[0]} small withdrawals: {sw_details}."
                )
            # 4. Collusion: Same customer updated by multiple employees.
            customer_updates = profile_df[profile_df['CUST_NUM'] == cust_num]
            unique_emps = customer_updates['CHG_BY'].unique()
            if len(unique_emps) > 1:
                alerts.append(("Collusion", "Cluster Alert"))
                record_reasoning.append(
                    f"Collusion: Customer updated by {len(unique_emps)} employees ({', '.join(unique_emps)})."
                )
            # 5. Legacy Customer: In absence of other alerts, large deposit post-update.
            if not alerts:
                large_deposits = post_txns[post_txns['TRANSACTION_AMOUNT'] >= LARGE_WITHDRAWAL_THRESHOLD]
                if not large_deposits.empty:
                    alerts.append(("Legacy Customer", "Low Risk Score"))
                    ld_details = "; ".join(
                        large_deposits.apply(lambda x: f"{x['TRANSACTION_DATE'].strftime('%Y-%m-%d')}({x['TRANSACTION_AMOUNT']})", axis=1)
                    )
                    record_reasoning.append(
                        f"Legacy Customer: Large deposit(s) detected: {ld_details}."
                    )
            
            # 6. Account Take Over (ATO): Additional checks focusing on ATO indicators.
            # Conditions may include:
            #    a) The employee's job title is not typical (expected: 'CSR').
            #    b) For EMAIL updates, the NEW_VALUE has an unexpected domain.
            #    c) A sudden significant shift in transaction pattern post-update.
            ato_flag = False
            ato_reasoning = []
            
            # a) Check employee job title.
            if emp_job_title.strip().upper() != "CSR":
                ato_flag = True
                ato_reasoning.append(f"Employee job title '{emp_job_title}' is not typical for profile updates (expected: CSR).")
            
            # b) For EMAIL updates, verify that the new email contains the expected domain.
            if profile_type == "EMAIL":
                new_email = row['NEW_VALUE']
                if "example.com" not in new_email.lower():
                    ato_flag = True
                    ato_reasoning.append(f"New email '{new_email}' has a suspicious domain (expected domain: example.com).")
            
            # c) Compare pre and post transaction patterns.
            #    For instance, if pre-update average is low but post-update average (especially withdrawals) is over 3 times higher.
            if pre_count > 0 and abs(post_avg) > 3 * abs(pre_avg) and post_avg < 0:
                ato_flag = True
                ato_reasoning.append(f"Post-update average transaction amount ({post_avg:.2f}) is over 3 times pre-update average ({pre_avg:.2f}).")
            
            if ato_flag:
                alerts.append(("Account Take Over", "Critical Risk Score"))
                record_reasoning.append("Account Take Over: " + " | ".join(ato_reasoning))
        
        # If no alert conditions were met, mark as "No Alert."
        if not alerts:
            alerts.append(("No Alert", "None"))
            record_reasoning.append("No Alert: No suspicious activity detected based on defined criteria.")
        
        audit_alerts.append(alerts)
        reasoning_list.append(" | ".join(record_reasoning))
    
    # ====================================================
    # STEP 6. AUGMENT THE PROFILE DATA WITH SUMMARIES, ALERTS, AND REASONING
    # ----------------------------------------------------
    profile_df['pre_txn_count']        = pre_txn_count_list
    profile_df['pre_txn_total']        = pre_txn_total_list
    profile_df['pre_txn_avg']          = pre_txn_avg_list
    profile_df['pre_txn_deposits']     = pre_txn_deposits_list
    profile_df['pre_txn_withdrawals']  = pre_txn_withdrawals_list
    profile_df['pre_txn_details']      = pre_txn_details_list

    profile_df['post_txn_count']       = post_txn_count_list
    profile_df['post_txn_total']       = post_txn_total_list
    profile_df['post_txn_avg']         = post_txn_avg_list
    profile_df['post_txn_deposits']    = post_txn_deposits_list
    profile_df['post_txn_withdrawals'] = post_txn_withdrawals_list
    profile_df['post_txn_details']     = post_txn_details_list

    profile_df['alerts']     = audit_alerts
    profile_df['alerts_str'] = profile_df['alerts'].apply(
        lambda alist: "; ".join([f"{scenario}: {score}" for scenario, score in alist])
    )
    profile_df['reasoning']  = reasoning_list
    
    # ====================================================
    # STEP 7. OUTPUT THE COMPLETE AUDIT TRAIL
    # ----------------------------------------------------
    audit_trail_df = profile_df.sort_values("PROFILE_UPDATE_DATE")
    print("\n--- Fraud Detection Audit Trail with Detailed Reasoning (Including Account Take Over) ---")
    print(audit_trail_df.to_string(index=False))
    
    audit_trail_df.to_csv("fraud_detection_audit_trail.csv", index=False)
    print("\nAudit trail written to 'fraud_detection_audit_trail.csv'.")

if __name__ == "__main__":
    main()



--- Fraud Detection Audit Trail with Detailed Reasoning (Including Account Take Over) ---
 CUST_NUM DATE_OF_BIRTH  CUST_TYPE PROFILE_UPDATE_DATE      PROFILE              NEW_VALUE              OLD_VALUE CHG_BY JOB_TITLE EMPLOYEE_JOB_TITLE  AGE  WEEK window_start window_end PROFILE_UPPER  pre_txn_count  pre_txn_total  pre_txn_avg  pre_txn_deposits  pre_txn_withdrawals  pre_txn_details  post_txn_count  post_txn_total  post_txn_avg  post_txn_deposits  post_txn_withdrawals                                        post_txn_details                              alerts                      alerts_str                                                                                                     reasoning
     1051    1981-03-21          0          2024-12-01        PHONE           555-265-8239           555-664-8007   E009       CSR                CSR   43    48   2024-12-01 2024-12-04         PHONE              0              0          0.0                 0                    0          

In [10]:
#!/usr/bin/env python3
import pandas as pd
from datetime import timedelta

def main():
    # ====================================================
    # STEP 1. LOAD THE DATASETS FROM THE LOCAL REPOSITORY
    # ----------------------------------------------------
    # Files:
    #   - CUST_CHG_RAW.csv: Customer profile change history (01/26/2025 to 02/01/2025)
    #   - EMP_PRF.csv: Employee information for those who changed customer profiles.
    #   - CUST_PRF.csv: Customer information (e.g., CUST_NUM, BIRTHDAY) for affected customers.
    #   - CUST_TRAN_HIST.csv: Transaction history from 21 days prior to 3 days after the profile change.
    cust_chg_raw = pd.read_csv('CUST_CHG_RAW.csv', parse_dates=['PROFILE_UPDATE_DATE'])
    cust_prf     = pd.read_csv('CUST_PRF.csv', parse_dates=['BIRTHDAY'])
    emp_prf      = pd.read_csv('EMP_PRF.csv')
    txn_df       = pd.read_csv('CUST_TRAN_HIST.csv', parse_dates=['TRANSACTION_DATE'])
    
    # Merge customer change data with customer profile information to obtain birthday.
    profile_df = pd.merge(cust_chg_raw, cust_prf, on='CUST_NUM', how='left')
    # Rename BIRTHDAY to DATE_OF_BIRTH (for consistency with subsequent calculations).
    profile_df.rename(columns={'BIRTHDAY': 'DATE_OF_BIRTH'}, inplace=True)
    # Merge with employee profile information. (Assuming EMP_PRF has an 'EMP_ID' column that matches CHG_BY.)
    profile_df = pd.merge(profile_df, emp_prf, left_on='CHG_BY', right_on='EMP_ID', how='left')
    
    # ====================================================
    # STEP 2. PRE-PROCESSING
    # ----------------------------------------------------
    # Compute the customer's age at the time of the profile update.
    profile_df['AGE'] = profile_df.apply(
        lambda row: (row['PROFILE_UPDATE_DATE'] - row['DATE_OF_BIRTH']).days // 365, axis=1
    )
    
    # Compute the ISO week number from PROFILE_UPDATE_DATE (to group employee activity).
    profile_df['WEEK'] = profile_df['PROFILE_UPDATE_DATE'].dt.isocalendar().week
    
    # Define the post-update transaction window (3 days after the profile update).
    profile_df['window_start'] = profile_df['PROFILE_UPDATE_DATE']
    profile_df['window_end']   = profile_df['PROFILE_UPDATE_DATE'] + pd.Timedelta(days=3)
    
    # Standardize the PROFILE field (e.g., EMAIL, PHONE, HOME ADDRESS).
    profile_df['PROFILE_UPPER'] = profile_df['PROFILE'].str.strip().str.upper()
    
    # ====================================================
    # STEP 3. DEFINE THRESHOLDS & CRITERIA
    # ----------------------------------------------------
    ELDERLY_AGE = 65
    LARGE_WITHDRAWAL_THRESHOLD = 10000   # For Clear Fraud: withdrawal amounts <= -$10,000.
    SMALL_WITHDRAWAL_THRESHOLD = 500     # Lower bound for small withdrawals (<= -$500).
    SMALL_WITHDRAWAL_COUNT_THRESHOLD = 3 # For Structuring: at least 3 small withdrawals.
    EMPLOYEE_HOME_ADDRESS_THRESHOLD = 5  # For Employee Workflow: 5+ HOME ADDRESS updates in a week.
    
    # ====================================================
    # STEP 4. IDENTIFY WHITELISTED EMPLOYEES (Employee Workflow)
    # ----------------------------------------------------
    # Whitelist employees who updated many HOME ADDRESS records in a given week.
    home_address_df = profile_df[profile_df['PROFILE_UPPER'] == 'HOME ADDRESS']
    emp_week_counts = home_address_df.groupby(['CHG_BY', 'WEEK']).size().reset_index(name='count')
    whitelisted_emp_week = set(
        tuple(x) for x in emp_week_counts[emp_week_counts['count'] >= EMPLOYEE_HOME_ADDRESS_THRESHOLD][['CHG_BY', 'WEEK']].to_numpy()
    )
    
    # ====================================================
    # STEP 5. PROCESS EACH PROFILE RECORD:
    #   - Compute pre (21 days) and post (3 days) transaction summaries.
    #   - Detect fraud scenarios (including Account Take Over conditions).
    #   - Build detailed reasoning for each record.
    # ----------------------------------------------------
    audit_alerts = []   # To store alerts for each record.
    reasoning_list = [] # To store detailed reasoning messages.
    
    # Lists to capture transaction summary statistics.
    pre_txn_count_list       = []
    pre_txn_total_list       = []
    pre_txn_avg_list         = []
    pre_txn_deposits_list    = []
    pre_txn_withdrawals_list = []
    pre_txn_details_list     = []
    
    post_txn_count_list       = []
    post_txn_total_list       = []
    post_txn_avg_list         = []
    post_txn_deposits_list    = []
    post_txn_withdrawals_list = []
    post_txn_details_list     = []
    
    for idx, row in profile_df.iterrows():
        cust_num     = row['CUST_NUM']
        update_dt    = row['PROFILE_UPDATE_DATE']
        week         = row['WEEK']
        employee     = row['CHG_BY']
        profile_type = row['PROFILE_UPPER']
        age          = row['AGE']
        # For ATO scenario, check employee job title (from the EMP_PRF merge). Default to "CSR" if missing.
        emp_job_title = row['EMPLOYEE_JOB_TITLE'] if 'EMPLOYEE_JOB_TITLE' in row and pd.notnull(row['EMPLOYEE_JOB_TITLE']) else "CSR"
        
        # ----- Define Transaction Windows -----
        # Pre-update window: 21 days before update (excluding the update date).
        pre_window_start = update_dt - pd.Timedelta(days=21)
        pre_window_end   = update_dt
        pre_txns = txn_df[
            (txn_df['CUST_NUM'] == cust_num) &
            (txn_df['TRANSACTION_DATE'] >= pre_window_start) &
            (txn_df['TRANSACTION_DATE'] < pre_window_end)
        ]
        
        # Post-update window: from update date through the next 3 days.
        post_window_start = update_dt
        post_window_end   = update_dt + pd.Timedelta(days=3)
        post_txns = txn_df[
            (txn_df['CUST_NUM'] == cust_num) &
            (txn_df['TRANSACTION_DATE'] >= post_window_start) &
            (txn_df['TRANSACTION_DATE'] <= post_window_end)
        ]
        
        # ----- Compute Pre-Transaction Summary Stats -----
        pre_count       = pre_txns.shape[0]
        pre_total       = pre_txns['TRANSACTION_AMOUNT'].sum() if pre_count > 0 else 0
        pre_avg         = pre_txns['TRANSACTION_AMOUNT'].mean() if pre_count > 0 else 0
        pre_deposits    = pre_txns[pre_txns['TRANSACTION_AMOUNT'] > 0].shape[0]
        pre_withdrawals = pre_txns[pre_txns['TRANSACTION_AMOUNT'] < 0].shape[0]
        if pre_count > 0:
            pre_details = "; ".join(
                pre_txns.apply(lambda x: f"{x['TRANSACTION_DATE'].strftime('%Y-%m-%d')}({x['TRANSACTION_AMOUNT']})", axis=1)
            )
        else:
            pre_details = ""
        
        pre_txn_count_list.append(pre_count)
        pre_txn_total_list.append(pre_total)
        pre_txn_avg_list.append(pre_avg)
        pre_txn_deposits_list.append(pre_deposits)
        pre_txn_withdrawals_list.append(pre_withdrawals)
        pre_txn_details_list.append(pre_details)
        
        # ----- Compute Post-Transaction Summary Stats -----
        post_count       = post_txns.shape[0]
        post_total       = post_txns['TRANSACTION_AMOUNT'].sum() if post_count > 0 else 0
        post_avg         = post_txns['TRANSACTION_AMOUNT'].mean() if post_count > 0 else 0
        post_deposits    = post_txns[post_txns['TRANSACTION_AMOUNT'] > 0].shape[0]
        post_withdrawals = post_txns[post_txns['TRANSACTION_AMOUNT'] < 0].shape[0]
        if post_count > 0:
            post_details = "; ".join(
                post_txns.apply(lambda x: f"{x['TRANSACTION_DATE'].strftime('%Y-%m-%d')}({x['TRANSACTION_AMOUNT']})", axis=1)
            )
        else:
            post_details = ""
        
        post_txn_count_list.append(post_count)
        post_txn_total_list.append(post_total)
        post_txn_avg_list.append(post_avg)
        post_txn_deposits_list.append(post_deposits)
        post_txn_withdrawals_list.append(post_withdrawals)
        post_txn_details_list.append(post_details)
        
        # ----- Detect Fraud Scenarios & Build Detailed Reasoning -----
        alerts = []
        record_reasoning = []  # List to accumulate reasoning messages.
        
        # 1. Employee Workflow (Whitelisted)
        if (employee, week) in whitelisted_emp_week:
            alerts.append(("Employee Workflow", "Whitelisted"))
            emp_count = emp_week_counts[
                (emp_week_counts['CHG_BY'] == employee) & (emp_week_counts['WEEK'] == week)
            ]['count'].values[0]
            record_reasoning.append(
                f"Employee Workflow: Employee {employee} performed {emp_count} HOME ADDRESS updates in week {week} (threshold = {EMPLOYEE_HOME_ADDRESS_THRESHOLD})."
            )
        else:
            # 2. Clear Fraud: Elderly (>=65) + Contact Change + Large Withdrawal (post-update)
            if age >= ELDERLY_AGE and profile_type in ['EMAIL', 'PHONE', 'HOME ADDRESS']:
                large_withdrawals = post_txns[post_txns['TRANSACTION_AMOUNT'] <= -LARGE_WITHDRAWAL_THRESHOLD]
                if not large_withdrawals.empty:
                    alerts.append(("Clear Fraud", "High Risk Score"))
                    lw_details = "; ".join(
                        large_withdrawals.apply(lambda x: f"{x['TRANSACTION_DATE'].strftime('%Y-%m-%d')}({x['TRANSACTION_AMOUNT']})", axis=1)
                    )
                    record_reasoning.append(
                        f"Clear Fraud: Elderly customer (age {age}) with {profile_type} update made large withdrawal(s): {lw_details}."
                    )
            # 3. Structuring: Multiple small withdrawals post-update.
            small_withdrawals = post_txns[
                (post_txns['TRANSACTION_AMOUNT'] <= -SMALL_WITHDRAWAL_THRESHOLD) &
                (post_txns['TRANSACTION_AMOUNT'] > -LARGE_WITHDRAWAL_THRESHOLD)
            ]
            if small_withdrawals.shape[0] >= SMALL_WITHDRAWAL_COUNT_THRESHOLD:
                alerts.append(("Structuring", "Medium Risk Score"))
                sw_details = "; ".join(
                    small_withdrawals.apply(lambda x: f"{x['TRANSACTION_DATE'].strftime('%Y-%m-%d')}({x['TRANSACTION_AMOUNT']})", axis=1)
                )
                record_reasoning.append(
                    f"Structuring: Found {small_withdrawals.shape[0]} small withdrawals: {sw_details}."
                )
            # 4. Collusion: Same customer updated by multiple employees.
            customer_updates = profile_df[profile_df['CUST_NUM'] == cust_num]
            unique_emps = customer_updates['CHG_BY'].unique()
            if len(unique_emps) > 1:
                alerts.append(("Collusion", "Cluster Alert"))
                record_reasoning.append(
                    f"Collusion: Customer updated by {len(unique_emps)} employees ({', '.join(unique_emps)})."
                )
            # 5. Legacy Customer: In absence of other alerts, large deposit post-update.
            if not alerts:
                large_deposits = post_txns[post_txns['TRANSACTION_AMOUNT'] >= LARGE_WITHDRAWAL_THRESHOLD]
                if not large_deposits.empty:
                    alerts.append(("Legacy Customer", "Low Risk Score"))
                    ld_details = "; ".join(
                        large_deposits.apply(lambda x: f"{x['TRANSACTION_DATE'].strftime('%Y-%m-%d')}({x['TRANSACTION_AMOUNT']})", axis=1)
                    )
                    record_reasoning.append(
                        f"Legacy Customer: Large deposit(s) detected: {ld_details}."
                    )
            
            # 6. Account Take Over (ATO): Additional checks for ATO indicators.
            # Conditions include:
            #    a) The employee's job title is not typical (expected: 'CSR').
            #    b) For EMAIL updates, the new email value does not contain the expected domain.
            #    c) A sudden significant shift in transaction pattern post-update.
            ato_flag = False
            ato_reasoning = []
            
            # a) Check employee job title.
            if emp_job_title.strip().upper() != "CSR":
                ato_flag = True
                ato_reasoning.append(f"Employee job title '{emp_job_title}' is not typical for profile updates (expected: CSR).")
            
            # b) For EMAIL updates, verify that the new email contains the expected domain.
            if profile_type == "EMAIL":
                new_email = row['NEW_VALUE']
                if "example.com" not in new_email.lower():
                    ato_flag = True
                    ato_reasoning.append(f"New email '{new_email}' has a suspicious domain (expected: example.com).")
            
            # c) Compare pre- and post-update transaction patterns.
            if pre_count > 0 and abs(post_avg) > 3 * abs(pre_avg) and post_avg < 0:
                ato_flag = True
                ato_reasoning.append(f"Post-update average transaction amount ({post_avg:.2f}) is over 3 times pre-update average ({pre_avg:.2f}).")
            
            if ato_flag:
                alerts.append(("Account Take Over", "Critical Risk Score"))
                record_reasoning.append("Account Take Over: " + " | ".join(ato_reasoning))
        
        # If no alert conditions are met, mark as "No Alert."
        if not alerts:
            alerts.append(("No Alert", "None"))
            record_reasoning.append("No Alert: No suspicious activity detected based on defined criteria.")
        
        audit_alerts.append(alerts)
        reasoning_list.append(" | ".join(record_reasoning))
    
    # ====================================================
    # STEP 6. AUGMENT THE PROFILE DATA WITH SUMMARIES, ALERTS, AND REASONING
    # ----------------------------------------------------
    profile_df['pre_txn_count']        = pre_txn_count_list
    profile_df['pre_txn_total']        = pre_txn_total_list
    profile_df['pre_txn_avg']          = pre_txn_avg_list
    profile_df['pre_txn_deposits']     = pre_txn_deposits_list
    profile_df['pre_txn_withdrawals']  = pre_txn_withdrawals_list
    profile_df['pre_txn_details']      = pre_txn_details_list

    profile_df['post_txn_count']       = post_txn_count_list
    profile_df['post_txn_total']       = post_txn_total_list
    profile_df['post_txn_avg']         = post_txn_avg_list
    profile_df['post_txn_deposits']    = post_txn_deposits_list
    profile_df['post_txn_withdrawals'] = post_txn_withdrawals_list
    profile_df['post_txn_details']     = post_txn_details_list

    profile_df['alerts']     = audit_alerts
    profile_df['alerts_str'] = profile_df['alerts'].apply(
        lambda alist: "; ".join([f"{scenario}: {score}" for scenario, score in alist])
    )
    profile_df['reasoning']  = reasoning_list
    
    # ====================================================
    # STEP 7. OUTPUT THE COMPLETE AUDIT TRAIL
    # ----------------------------------------------------
    audit_trail_df = profile_df.sort_values("PROFILE_UPDATE_DATE")
    print("\n--- Fraud Detection Audit Trail with Detailed Reasoning (Including Account Take Over) ---")
    print(audit_trail_df.to_string(index=False))
    
    audit_trail_df.to_csv("fraud_detection_audit_trail.csv", index=False)
    print("\nAudit trail written to 'fraud_detection_audit_trail.csv'.")

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'CUST_CHG_RAW.csv'